In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
train_data = pd.read_csv('Train_Data.csv')
test_data = pd.read_csv('Test_Data.csv')
train_data.head()

,date,campaign,adgroup,ad,impressions,clicks,cost,conversions,revenue
0,01-08-2020,campaign 1,adgroup 1,ad 1,24,6,0.08,0,0.00
1,01-08-2020,campaign 1,adgroup 2,ad 1,1,0,0.00,0,0.00
2,01-08-2020,campaign 1,adgroup 3,ad 1,13,4,0.04,0,0.00
3,01-08-2020,campaign 1,adgroup 4,ad 1,5,4,0.08,0,0.00
4,01-08-2020,campaign 1,adgroup 1,ad 2,247,126,1.29,4,925.71


In [3]:
train_data.corr()

,impressions,clicks,cost,conversions,revenue
impressions,1.000000,0.987514,0.682811,0.822149,0.759671
clicks,0.987514,1.000000,0.719925,0.856941,0.798282
cost,0.682811,0.719925,1.000000,0.672202,0.581662
conversions,0.822149,0.856941,0.672202,1.000000,0.964096
revenue,0.759671,0.798282,0.581662,0.964096,1.000000


In [4]:
train_data.drop('date', axis=1, inplace=True)
train_data.drop('campaign', axis=1, inplace=True)
test_data.drop('date', axis=1, inplace=True)
test_data.drop('campaign', axis=1, inplace=True)
train_data.drop('ad', axis=1, inplace=True)
test_data.drop('ad', axis=1, inplace=True)

In [5]:
train_data['CPA'] = train_data['cost']/train_data['conversions']
test_data['CPA'] = test_data['cost']/test_data['conversions']

In [6]:
train_data.replace([np.inf, -np.inf], 0, inplace=True)
test_data.replace([np.inf, -np.inf], 0, inplace=True)
train_data.fillna(0, inplace=True)
test_data.fillna(0, inplace=True)
train_data.head()

,adgroup,impressions,clicks,cost,conversions,revenue,CPA
0,adgroup 1,24,6,0.08,0,0.00,0.0000
1,adgroup 2,1,0,0.00,0,0.00,0.0000
2,adgroup 3,13,4,0.04,0,0.00,0.0000
3,adgroup 4,5,4,0.08,0,0.00,0.0000
4,adgroup 1,247,126,1.29,4,925.71,0.3225


In [7]:
train_data.corr()

,impressions,clicks,cost,conversions,revenue,CPA
impressions,1.000000,0.987514,0.682811,0.822149,0.759671,0.363711
clicks,0.987514,1.000000,0.719925,0.856941,0.798282,0.345763
cost,0.682811,0.719925,1.000000,0.672202,0.581662,0.372536
conversions,0.822149,0.856941,0.672202,1.000000,0.964096,0.157083
revenue,0.759671,0.798282,0.581662,0.964096,1.000000,0.125732
CPA,0.363711,0.345763,0.372536,0.157083,0.125732,1.000000


In [8]:
train_data = pd.get_dummies(train_data)
test_data = pd.get_dummies(test_data)
train_data.head()

,impressions,clicks,cost,conversions,revenue,CPA,adgroup_adgroup 1,adgroup_adgroup 2,adgroup_adgroup 3,adgroup_adgroup 4
0,24,6,0.08,0,0.00,0.0000,1,0,0,0
1,1,0,0.00,0,0.00,0.0000,0,1,0,0
2,13,4,0.04,0,0.00,0.0000,0,0,1,0
3,5,4,0.08,0,0.00,0.0000,0,0,0,1
4,247,126,1.29,4,925.71,0.3225,1,0,0,0


In [9]:
X_train = train_data.drop(['revenue'], axis='columns')
y_train = train_data['revenue']
X_test = test_data

In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_train_scaled

X_test_scaled = scaler.transform(X_test)
X_test_scaled

array([[-1.29709697e-04,  6.83229814e-02,  9.22352941e-02, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-2.56331069e-04,  1.24223602e-02,  2.25882353e-02, ...,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [-2.93390982e-04,  2.82326369e-03,  5.64705882e-03, ...,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       ...,
       [-2.65596047e-04,  1.41163185e-02,  2.07058824e-02, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-2.90302656e-04,  5.08187465e-03,  7.52941176e-03, ...,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [-2.06917851e-04,  3.78317335e-02,  5.08235294e-02, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [11]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(100, activation="relu"),
  tf.keras.layers.Dense(50),
  tf.keras.layers.Dense(1),
])

# 2. Compile the model
model.compile( loss = tf.keras.losses.mae,
               optimizer = tf.keras.optimizers.Adam(),
               metrics = tf.keras.metrics.RootMeanSquaredError())

# 3. Fit the model
model.fit(X_train_scaled, y_train, epochs= 100)

Epoch 1/100
143/143 [==============================] - 1s 2ms/step - loss: 231.8077 - root_mean_squared_error: 1115.7909
Epoch 2/100
143/143 [==============================] - 0s 2ms/step - loss: 230.2145 - root_mean_squared_error: 1110.8828
Epoch 3/100
143/143 [==============================] - 0s 2ms/step - loss: 217.1076 - root_mean_squared_error: 1071.7722
Epoch 4/100
143/143 [==============================] - 0s 2ms/step - loss: 188.6838 - root_mean_squared_error: 980.2556
Epoch 5/100
143/143 [==============================] - 0s 2ms/step - loss: 168.6860 - root_mean_squared_error: 887.0896
Epoch 6/100
143/143 [==============================] - 0s 2ms/step - loss: 159.8581 - root_mean_squared_error: 820.3170
Epoch 7/100
143/143 [==============================] - 0s 2ms/step - loss: 154.9954 - root_mean_squared_error: 779.2055
Epoch 8/100
143/143 [==============================] - 0s 2ms/step - loss: 150.3698 - root_mean_squared_error: 745.5649
Epoch 9/100
143/143 [================

In [12]:
preds = model.predict(X_test_scaled)

In [13]:
preds = preds.astype('int64')
preds

array([[ 180],
       [  13],
       [   0],
       [  -1],
       [   0],
       [   0],
       [ 116],
       [   0],
       [ 186],
       [   0],
       [ 469],
       [  14],
       [  11],
       [ 343],
       [  -1],
       [ 189],
       [  14],
       [ 182],
       [  14],
       [ 170],
       [   0],
       [  14],
       [  -1],
       [ 461],
       [   0],
       [   0],
       [ 633],
       [ 170],
       [   1],
       [   0],
       [ 483],
       [  10],
       [1005],
       [ 354],
       [ 168],
       [   0],
       [ 185],
       [  14],
       [   0],
       [   9],
       [  11],
       [  18],
       [   0],
       [  19],
       [  22],
       [   0],
       [   9],
       [   8],
       [  13],
       [  11],
       [   0],
       [   9],
       [  19],
       [ 348],
       [ 163],
       [ 124],
       [  -1],
       [   7],
       [   0],
       [1061],
       [   0],
       [ 347],
       [   0],
       [ 287],
       [   0],
       [  30],
       [ 5

In [14]:
preds[preds < 30] = 0
preds

array([[ 180],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [ 116],
       [   0],
       [ 186],
       [   0],
       [ 469],
       [   0],
       [   0],
       [ 343],
       [   0],
       [ 189],
       [   0],
       [ 182],
       [   0],
       [ 170],
       [   0],
       [   0],
       [   0],
       [ 461],
       [   0],
       [   0],
       [ 633],
       [ 170],
       [   0],
       [   0],
       [ 483],
       [   0],
       [1005],
       [ 354],
       [ 168],
       [   0],
       [ 185],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [ 348],
       [ 163],
       [ 124],
       [   0],
       [   0],
       [   0],
       [1061],
       [   0],
       [ 347],
       [   0],
       [ 287],
       [   0],
       [  30],
       [ 5

In [15]:
prediction = pd.DataFrame(preds, columns=['revenue']).to_csv('prediction.csv', index=False)